In [ ]:
# Evaluation of feature selection methods in estimation of precipitation based on deep learning artificial neural networks
# by
# Mohammad Taghi Sattari, Anca Avram, Halit Apaydin, Oliviu Matei
#
!pip install HydroErr
import HydroErr as HEEM
import numpy 
import pandas as pd
from pandas import read_csv
import math
import keras
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Activation
from sklearn.metrics import mean_squared_error
from keras.layers import BatchNormalization, Dropout
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
AI_Method = "/content/gdrive/My Drive/Colab Notebooks/Hamadan/H-Sarabi-Allb"
Var_LRs=[1e-2,1e-3]#,1e-3]#,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]
Var_Decays=[1e-2,1e-3]#,1e-3]
Var_epochs=[5,10,50,100,500,1000]#,100,300,500,1000,1500,2000]
 
Streamflow=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Hamadan/Hamadan.csv', delimiter=';')

In [ ]:
import numpy as np
x= Streamflow.drop('Sarabi',axis=1)
Y= Streamflow['Sarabi']
x= x.drop('Month',axis=1)

'''
x= x.drop('Khosroabad',axis=1)
x= x.drop('Namileh',axis=1)
x= x.drop('Pihan',axis=1)
x= x.drop('Kheyirabad',axis=1)

x= x.drop('Sarabi',axis=1)
x= x.drop('Vasaj',axis=1)
x= x.drop('Babapirali',axis=1) 
x= x.drop('Varayineh',axis=1) 
'''

x.head()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Varayineh   300 non-null    float64
 1   Kheyirabad  300 non-null    float64
 2   Khosroabad  300 non-null    float64
 3   Namileh     300 non-null    float64
 4   Pihan       300 non-null    float64
 5   Vasaj       300 non-null    float64
 6   Babapirali  300 non-null    float64
dtypes: float64(7)
memory usage: 16.5 KB


In [ ]:
X=np.array(x)
y=np.array(Y)
print (X.shape[1])

7


In [ ]:
test_size = int(len(X) * 0.30)
valid_size = 0# int(len(X) * 0.15)
train_size= len(X) - (valid_size+test_size)
y_train, y_valid, y_test = y[0:train_size], y[train_size:train_size+valid_size], y[-test_size:]
X_train, X_valid, X_test = X[0:train_size], X[train_size:train_size+valid_size], X[train_size+valid_size:]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
print (y_train.shape, y_valid.shape, y_test.shape)

(210,) (0,) (90,)


In [ ]:
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as geek

In [ ]:
#def create_ANN():
'''
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=2, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0002), input_shape=(X.shape[1],1)))
model.add(Conv1D(filters=16, kernel_size=2, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0002)))
model.add(Dropout(0.028))
model.add(MaxPooling1D(pool_size=1))
#model.add(Flatten())            
#model.add(MaxPooling1D(pool_size=1))
#model.add(Flatten())
#model.add(Conv1D(filters=64, kernel_size=1, activation='softmax'))  
model.add(Flatten())         
model.add(Dense(2, activation='relu'))
model.add(Dense(1))

'''
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=2, activation='sigmoid', input_shape=(X.shape[1],1)))
model.add(Conv1D(filters=16, kernel_size=1, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())         
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [ ]:
startTime = datetime.now()
fileOfSummary = open(AI_Method+'_Summary_'+str(datetime.now()).replace(":", ".")+'.csv', "w")
fileOfSummary.write("Method,LR,DE,Epoch,RunTime,CCTr,CCTt,NSTr,NSTt,WITr,WITt,RMSETr,RMSETt,MAETr,MAETt"+str(X.shape[1])+"\n")

84

In [ ]:
for e_LR in Var_LRs:
    for e_decay in Var_Decays:
        for e_epoch in Var_epochs:
            
            #model = create_ANN()
            
            optimizer = keras.optimizers.Adam(lr=e_LR, decay=e_decay)
            model.compile(optimizer=optimizer,loss='mean_squared_error')
        
            print(" ")
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            history = model.fit(X_train, y_train, epochs=e_epoch, batch_size=256, verbose=0, validation_data=(X_valid, y_valid), shuffle=False)
            hist_df = pd.DataFrame(history.history) 
            Time_elasped= datetime.now() - startTime
            print('\nTime elapsed: ', Time_elasped)
 
            Train = model.predict(X_train)
            Test = model.predict(X_test)
 
            FileName=AI_Method+'-LR'+str(e_LR)+'-DE'+str(e_decay)+'-'+str(e_epoch)
            np.savetxt(FileName+'_Train.csv', Train)
            np.savetxt(FileName+'_Test.csv', Test)
            Train = geek.asarray(Train)
            Test = geek.asarray(Test)
 
            Train = Train.reshape(Train.shape[0])
            Test = Test.reshape(Test.shape[0])
 
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            print("Train ==>")
            CC_Train=np.corrcoef(y_train,Train)
            print("CC_Train = %.3f" %CC_Train[0,1])
 
            NSTr=HEEM.nse(Train, y_train) 
            print("NSTr = %.2f" %NSTr)
 
            WITr=HEEM.d(Train, y_train)     
            print("WITr = %.2f" %WITr)

            rootMeanSquaredErrorTr = sqrt(mean_squared_error(y_train, Train))
            print("RMSE = %.2f" % rootMeanSquaredErrorTr)
            MAETr=mean_absolute_error(y_train, Train)
            print("MAE = %.2f" %  MAETr)
 
            print("Test ======================>")
            CC_Test=np.corrcoef(y_test,Test)
            print("CC_Test = %.3f" %CC_Test[0,1])
 
            NSTt=HEEM.nse(Test, y_test)
            print("NSTt = %.2f" %NSTt)
 
            WITt=HEEM.d(Test, y_test) 
            print("WITr = %.2f" %WITr)

            rootMeanSquaredErrorTt = sqrt(mean_squared_error(y_test, Test))
            print("RMSE = %.2f" % rootMeanSquaredErrorTt)
            MAETt=mean_absolute_error(y_test, Test)
            print("MAE = %.2f" %  MAETt)
 
            fileOfSummary.write(AI_Method+','+str(e_LR)+','+str(e_decay)+','+str(e_epoch)+','+str(Time_elasped)+','
              +str(CC_Train[0,1])+','+str(CC_Test[0,1])+','
              +str(NSTr)+','+str(NSTt)+','
              +str(WITr)+','+str(WITt)+','
              +str(rootMeanSquaredErrorTr)+','+str(rootMeanSquaredErrorTt)+','
              +str(MAETr)+','+str(MAETt)+'\n')

            keras.backend.clear_session()
            model.reset_states() 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


 
 
LRs: 0.01
Decays: 0.01
epochs: 5

Time elapsed:  0:00:17.043529
 
LRs: 0.01
Decays: 0.01
epochs: 5
Train ==>
CC_Train = 0.731
NSTr = -0.21
WITr = 0.39
RMSE = 43.65
MAE = 30.63
Test ======================>
CC_Test = 0.669
NSTt = -0.19
WITr = 0.39
RMSE = 58.41
MAE = 37.03
 
 
LRs: 0.01
Decays: 0.01
epochs: 10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Time elapsed:  0:00:18.471701
 
LRs: 0.01
Decays: 0.01
epochs: 10
Train ==>
CC_Train = 0.762
NSTr = 0.10
WITr = 0.21
RMSE = 37.78
MAE = 30.49
Test ======================>
CC_Test = 0.685
NSTt = 0.07
WITr = 0.21
RMSE = 51.62
MAE = 36.75
 
 
LRs: 0.01
Decays: 0.01
epochs: 50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


...


Time elapsed:  0:02:15.874498
 
LRs: 0.001
Decays: 0.001
epochs: 100
Train ==>
CC_Train = 0.985
NSTr = 0.97
WITr = 0.99
RMSE = 6.99
MAE = 4.61
Test ======================>
CC_Test = 0.964
NSTt = 0.93
WITr = 0.99
RMSE = 14.55
MAE = 8.09
 
 
LRs: 0.001
Decays: 0.001
epochs: 500


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Time elapsed:  0:02:24.182890
 
LRs: 0.001
Decays: 0.001
epochs: 500
Train ==>
CC_Train = 0.985
NSTr = 0.97
WITr = 0.99
RMSE = 7.03
MAE = 4.61
Test ======================>
CC_Test = 0.966
NSTt = 0.93
WITr = 0.99
RMSE = 14.35
MAE = 8.04
 
 
LRs: 0.001
Decays: 0.001
epochs: 1000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Time elapsed:  0:02:45.753650
 
LRs: 0.001
Decays: 0.001
epochs: 1000
Train ==>
CC_Train = 0.986
NSTr = 0.97
WITr = 0.99
RMSE = 6.75
MAE = 4.48
Test ======================>
CC_Test = 0.965
NSTt = 0.93
WITr = 0.99
RMSE = 14.20
MAE = 8.20


In [ ]:
import io
buf = io.StringIO()
x.info(buf=buf)
s = buf.getvalue()

fileOfSummary.write(s)
fileOfSummary.close()

print("Finished")

Finished
